# New York Taxi Tip Prediction

### Importing Libraries

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
train = pd.read_csv("taxi-train.csv")
test = pd.read_csv("taxi-test.csv")

In [10]:
train.head(5)

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,rate_code,passenger_count,trip_distance,payment_type,fare_amount,tip_amount,tip_paid
0,CMT,2013-11-03 18:11:18+00:00,2013-11-03 18:18:43+00:00,-73.971879,40.756228,-73.971428,40.764175,1.0,1,0.80,CSH,6.5,0.00,0
1,CMT,2010-06-30 17:30:34+00:00,2010-06-30 17:40:44+00:00,-73.977850,40.738361,-73.977073,40.754229,1.0,1,1.50,Cre,6.9,1.88,1
2,CMT,2010-03-03 10:10:25+00:00,2010-03-03 10:19:38+00:00,-73.974089,40.762479,-73.990669,40.745668,1.0,1,1.60,Cre,6.9,1.00,1
3,VTS,2012-04-17 11:51:00+00:00,2012-04-17 12:07:00+00:00,-73.979920,40.781045,-73.970415,40.765608,1.0,1,1.70,CRD,9.7,1.94,1
4,VTS,2012-03-30 14:26:00+00:00,2012-03-30 14:36:00+00:00,-73.972975,40.761610,-73.955615,40.764290,1.0,2,1.36,CSH,6.9,0.00,0
